In [13]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from category_encoders import OrdinalEncoder
from sklearn.ensemble import GradientBoostingClassifier
from utils import extract_dates, get_val_scores
from sklearn.model_selection import cross_val_score, train_test_split
import xgboost as xgb

In [15]:
import sys
!{sys.executable} -m pip install xgboost

In [16]:
df = pd.read_csv('/Users/harleyhoffmann/dat-02-22/ClassMaterial/Unit3/data/ks2.csv', parse_dates=['deadline', 'launched' ])

In [17]:
df = extract_dates(df)
#allows us to rip out every last time part

/Users/harleyhoffmann/dat-02-22/ClassMaterial/Unit3/Class14/utils.py:28: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df[col_name] = getattr(df[col].dt, part)


In [18]:
df.columns

Index(['ID', 'name', 'category', 'main_category', 'currency', 'deadline',
       'launched', 'state', 'country', 'goal', 'deadline_dayofweek',
       'deadline_dayofyear', 'deadline_days_in_month', 'deadline_is_leap_year',
       'deadline_is_month_end', 'deadline_is_month_start',
       'deadline_is_quarter_end', 'deadline_is_quarter_start',
       'deadline_is_year_end', 'deadline_is_year_start', 'deadline_quarter',
       'deadline_week', 'deadline_weekofyear', 'deadline_day', 'deadline_hour',
       'deadline_minute', 'deadline_month', 'deadline_year',
       'launched_dayofweek', 'launched_dayofyear', 'launched_days_in_month',
       'launched_is_leap_year', 'launched_is_month_end',
       'launched_is_month_start', 'launched_is_quarter_end',
       'launched_is_quarter_start', 'launched_is_year_end',
       'launched_is_year_start', 'launched_quarter', 'launched_week',
       'launched_weekofyear', 'launched_day', 'launched_hour',
       'launched_minute', 'launched_month', 'laun

In [19]:
X = df.drop(['launched', 'deadline'], axis=1)
y = df['state']

In [20]:
pipe = make_pipeline(OrdinalEncoder(), xgb.XGBClassifier())

In [21]:
val_scores = get_val_scores(pipe, X, y, return_test_score=True, random_state=1985, stratify=True)
#get a quick look of validation and test scores

/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, 

/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, 

AttributeError: dlsym(0x7fb197609da0, XGDMatrixSetDenseInfo): symbol not found

In [ ]:
feats = pd.DataFrame({'Features':pipe[-2].get_feature_names(), 'Importances':pipe[-1].feature_importances_}).sort_values(by='Importances', ascending=False)
feats

In [25]:
#how long is your campaign
df['duration'] = (df['deadline'] - df['launched']).dt.days

In [26]:
# get the average goal for each category
category_avgs = df.groupby('category')[['goal']].mean().reset_index().rename({'goal': 'category_goal_avg'}, axis=1)

In [27]:
#mer
df = df.merge(category_avgs, on='category')


In [28]:
df['cat_goal_pct'] = df['goal'] / df['category_goal_avg']
#get the percent of category average of the overall total

In [29]:
df[['goal', 'category_goal_avg', 'cat_goal_pct']].head()

,goal,category_goal_avg,cat_goal_pct
0,1533.95,5213.996468,0.294199
1,6060.97,5213.996468,1.162442
2,2000.00,5213.996468,0.383583
3,10000.00,5213.996468,1.917915
4,757.52,5213.996468,0.145286


In [ ]:
#Let's re-declare and run
X = df.drop(['launched', 'deadline'], axis=1)
y = df['state']

In [ ]:
scores = get_val_scores(pipe, X, y, random_state=1985, stratify=True, use_kfold=False)

In [11]:
mod = xgb.XGBClassifier()

In [12]:
mod.get_params()

{'objective': 'binary:logistic',
 'use_label_encoder': True,
 'base_score': None,
 'booster': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'gamma': None,
 'gpu_id': None,
 'importance_type': 'gain',
 'interaction_constraints': None,
 'learning_rate': None,
 'max_delta_step': None,
 'max_depth': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [22]:
df['name_length'] = df['name'].str.len()

In [23]:
df

,ID,name,category,main_category,currency,deadline,launched,state,country,goal,...,launched_week,launched_weekofyear,launched_day,launched_hour,launched_minute,launched_month,launched_year,deadline_history_days,launched_history_days,name_length
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,2015-08-11 12:12:28,failed,GB,1533.95,...,33,33,11,12,12,8,2015,2350,16658,31.0
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,2017-09-02 04:43:57,failed,US,30000.00,...,35,35,2,4,43,9,2017,3104,17411,45.0
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,2013-01-12 00:20:50,failed,US,45000.00,...,2,2,12,0,20,1,2013,1395,15716,14.0
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,2012-03-17 03:24:11,failed,US,5000.00,...,11,11,17,3,24,3,2012,1079,15416,49.0
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,2015-07-04 08:35:03,failed,US,19500.00,...,27,27,4,8,35,7,2015,2309,16620,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370449,999976400,ChknTruk Nationwide Charity Drive 2014 (Canceled),Documentary,Film & Video,USD,2014-10-17,2014-09-17 02:35:30,failed,US,50000.00,...,38,38,17,2,35,9,2014,1993,16330,49.0
370450,999977640,The Tribe,Narrative Film,Film & Video,USD,2011-07-19,2011-06-22 03:35:14,failed,US,1500.00,...,25,25,22,3,35,6,2011,807,15147,9.0
370451,999986353,Walls of Remedy- New lesbian Romantic Comedy f...,Narrative Film,Film & Video,USD,2010-08-16,2010-07-01 19:40:30,failed,US,15000.00,...,26,26,1,19,40,7,2010,470,14791,71.0
370452,999987933,BioDefense Education Kit,Technology,Technology,USD,2016-02-13,2016-01-13 18:13:53,failed,US,15000.00,...,2,2,13,18,13,1,2016,2477,16813,24.0


In [34]:
df['name_upper_case'] = df['name'].str.isupper().astype(bool)
df['name_lower_case'] = df['name'].str.islower().astype(bool)

In [35]:
df

,ID,name,category,main_category,currency,deadline,launched,state,country,goal,...,launched_month,launched_year,deadline_history_days,launched_history_days,name_length,duration,category_goal_avg,cat_goal_pct,name_upper_case,name_lower_case
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,2015-08-11 12:12:28,failed,GB,1533.95,...,8,2015,2350,16658,31.0,58,5213.996468,0.294199,False,False
1,1001028167,Steel Cathedrals- Short poems for the digital ...,Poetry,Publishing,GBP,2013-07-28,2013-06-28 23:17:04,failed,GB,6060.97,...,6,2013,1547,15884,50.0,29,5213.996468,1.162442,False,False
2,1001468086,Bass River Press,Poetry,Publishing,USD,2015-04-05,2015-03-06 19:58:58,failed,US,2000.00,...,3,2015,2163,16500,16.0,29,5213.996468,0.383583,False,False
3,1001991458,Poems For Apostates & tales of a young Sciento...,Poetry,Publishing,USD,2014-05-31,2014-05-01 17:47:49,failed,US,10000.00,...,5,2014,1854,16191,52.0,29,5213.996468,1.917915,False,False
4,1002519316,"Your poem, by me.",Poetry,Publishing,CAD,2015-10-31,2015-10-01 20:00:32,failed,CA,757.52,...,10,2015,2372,16709,17.0,29,5213.996468,0.145286,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370449,93590450,The Friendship Club,Residencies,Dance,USD,2014-09-03,2014-08-04 04:08:38,successful,US,3000.00,...,8,2014,1949,16286,19.0,29,11802.966957,0.254173,False,False
370450,948299537,rLand,Residencies,Dance,CAD,2014-10-18,2014-08-19 20:09:33,failed,CA,446030.33,...,8,2014,1994,16301,5.0,59,11802.966957,37.789679,False,False
370451,952585464,Lxs Desaparecidxs,Residencies,Dance,USD,2017-08-11,2017-07-11 23:12:32,successful,US,3000.00,...,7,2017,3022,17358,17.0,30,11802.966957,0.254173,False,False
370452,97395296,GRADUADOS DANZA 2017,Residencies,Dance,MXN,2017-07-07,2017-06-12 18:35:20,failed,MX,20349.80,...,6,2017,2987,17329,20.0,24,11802.966957,1.724126,True,False


In [36]:
df['name_is_title'] = df['name'].str.istitle().astype(bool)

In [37]:
df

,ID,name,category,main_category,currency,deadline,launched,state,country,goal,...,launched_year,deadline_history_days,launched_history_days,name_length,duration,category_goal_avg,cat_goal_pct,name_upper_case,name_lower_case,name_is_title
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,2015-08-11 12:12:28,failed,GB,1533.95,...,2015,2350,16658,31.0,58,5213.996468,0.294199,False,False,False
1,1001028167,Steel Cathedrals- Short poems for the digital ...,Poetry,Publishing,GBP,2013-07-28,2013-06-28 23:17:04,failed,GB,6060.97,...,2013,1547,15884,50.0,29,5213.996468,1.162442,False,False,False
2,1001468086,Bass River Press,Poetry,Publishing,USD,2015-04-05,2015-03-06 19:58:58,failed,US,2000.00,...,2015,2163,16500,16.0,29,5213.996468,0.383583,False,False,True
3,1001991458,Poems For Apostates & tales of a young Sciento...,Poetry,Publishing,USD,2014-05-31,2014-05-01 17:47:49,failed,US,10000.00,...,2014,1854,16191,52.0,29,5213.996468,1.917915,False,False,False
4,1002519316,"Your poem, by me.",Poetry,Publishing,CAD,2015-10-31,2015-10-01 20:00:32,failed,CA,757.52,...,2015,2372,16709,17.0,29,5213.996468,0.145286,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370449,93590450,The Friendship Club,Residencies,Dance,USD,2014-09-03,2014-08-04 04:08:38,successful,US,3000.00,...,2014,1949,16286,19.0,29,11802.966957,0.254173,False,False,True
370450,948299537,rLand,Residencies,Dance,CAD,2014-10-18,2014-08-19 20:09:33,failed,CA,446030.33,...,2014,1994,16301,5.0,59,11802.966957,37.789679,False,False,False
370451,952585464,Lxs Desaparecidxs,Residencies,Dance,USD,2017-08-11,2017-07-11 23:12:32,successful,US,3000.00,...,2017,3022,17358,17.0,30,11802.966957,0.254173,False,False,True
370452,97395296,GRADUADOS DANZA 2017,Residencies,Dance,MXN,2017-07-07,2017-06-12 18:35:20,failed,MX,20349.80,...,2017,2987,17329,20.0,24,11802.966957,1.724126,True,False,False


In [38]:
df['name_exclamation'] = df['name'].str.contains('!').astype(bool)

In [39]:
df

,ID,name,category,main_category,currency,deadline,launched,state,country,goal,...,deadline_history_days,launched_history_days,name_length,duration,category_goal_avg,cat_goal_pct,name_upper_case,name_lower_case,name_is_title,name_exclamation
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,2015-08-11 12:12:28,failed,GB,1533.95,...,2350,16658,31.0,58,5213.996468,0.294199,False,False,False,False
1,1001028167,Steel Cathedrals- Short poems for the digital ...,Poetry,Publishing,GBP,2013-07-28,2013-06-28 23:17:04,failed,GB,6060.97,...,1547,15884,50.0,29,5213.996468,1.162442,False,False,False,False
2,1001468086,Bass River Press,Poetry,Publishing,USD,2015-04-05,2015-03-06 19:58:58,failed,US,2000.00,...,2163,16500,16.0,29,5213.996468,0.383583,False,False,True,False
3,1001991458,Poems For Apostates & tales of a young Sciento...,Poetry,Publishing,USD,2014-05-31,2014-05-01 17:47:49,failed,US,10000.00,...,1854,16191,52.0,29,5213.996468,1.917915,False,False,False,False
4,1002519316,"Your poem, by me.",Poetry,Publishing,CAD,2015-10-31,2015-10-01 20:00:32,failed,CA,757.52,...,2372,16709,17.0,29,5213.996468,0.145286,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370449,93590450,The Friendship Club,Residencies,Dance,USD,2014-09-03,2014-08-04 04:08:38,successful,US,3000.00,...,1949,16286,19.0,29,11802.966957,0.254173,False,False,True,False
370450,948299537,rLand,Residencies,Dance,CAD,2014-10-18,2014-08-19 20:09:33,failed,CA,446030.33,...,1994,16301,5.0,59,11802.966957,37.789679,False,False,False,False
370451,952585464,Lxs Desaparecidxs,Residencies,Dance,USD,2017-08-11,2017-07-11 23:12:32,successful,US,3000.00,...,3022,17358,17.0,30,11802.966957,0.254173,False,False,True,False
370452,97395296,GRADUADOS DANZA 2017,Residencies,Dance,MXN,2017-07-07,2017-06-12 18:35:20,failed,MX,20349.80,...,2987,17329,20.0,24,11802.966957,1.724126,True,False,False,False


In [40]:
#Let's re-declare and run
X = df.drop(['launched', 'deadline'], axis=1)
y = df['state']

In [41]:
scores = get_val_scores(pipe, X, y, random_state=1985, stratify=True, use_kfold=False)


/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


AttributeError: dlsym(0x7fb197609da0, XGDMatrixSetDenseInfo): symbol not found

In [ ]:
# so we'll drop
#df.drop(['name_length', 'name_upper_case', 'name_lower_case', 'name_is_title', 'name_exclamation'], axis=1, inplace=True)
#If we wanted to check if they were actually important we would just increse the amount of boosting rounds

In [ ]:
useless_cols = feats[feats['Importance'] == 0]
#filtering out the bad feats to improve fitting time


In [ ]:
useless_cols['Columns']